# Proteomics dimethyl-labeling

# Data Processing

This notebook contains data processing steps for dymethil-labeling proteomic workflow.

Demonstration proteomic dataset is available from [PRIDE PXD000497](https://www.ebi.ac.uk/pride/archive/projects/PXD000497) as BRAF_Wholeproteome_MaxQuant_Output.zip.

MaxQuant output proteinGroups.txt is used as input to generate processed '.csv' and '.pickle' files for the subsequent data analysis. Information relative to the kind of experiment are contained in design.csv.

In [1]:
%matplotlib inline
%load_ext autoreload

import matplotlib.pyplot as plt
plt.style.use('ggplot')
import os
from datetime import datetime as dt
dtstr = dt.strftime(dt.now(), "%Y%m%d")
import numpy as np
import pandas as pd
import padua
import padua.visualize as viz

Initial steps include uploading table files containing protein-data and experimental design, and apply standard filters steps used for MaxQuant tables.

In [2]:
# Load input files
PROTEIN_FILE = r'proteinGroups.txt'
DESIGN_FILE = r'design.csv'

In [3]:
# Define where to save output files
SOURCE_FOLDER = os.path.dirname(PROTEIN_FILE)
BASENAME = os.path.splitext(os.path.basename(PROTEIN_FILE))[0]

In [4]:
# Read input file proteinGroups.txt from MaxQuant
df = padua.io.read_maxquant(PROTEIN_FILE)
df.shape

C:\Anaconda2\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:2903: DtypeWarning: Columns (132) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


(5903, 142)

In [5]:
%autoreload 2

# Apply standard filter steps and save filtered output
df = padua.filters.remove_reverse(df)
df = padua.filters.remove_contaminants(df)
df = padua.filters.remove_only_identified_by_site(df)
df.to_csv( os.path.join(SOURCE_FOLDER,  BASENAME + '-after-filters.csv'))
df.shape

(5718, 142)

In [6]:
%autoreload 2
df['Proteins'].sum()

6449

In [7]:
%autoreload 2
df['Unique peptides'].sum()

59036

The filtered DataFrame is subsequently processed in order to select specific columns for the the subsequent Statistical Analysis workflow.

In [8]:
%autoreload 2

# Prepare protein DataFrame for next statistical analysis
df = padua.process.transform_expression_columns(df, fn=np.log2, prefix="Ratio M/L normalized")
df = padua.process.transform_expression_columns(df, fn=np.log2, prefix="Ratio H/L normalized")
df = padua.process.transform_expression_columns(df, fn=np.log2, prefix="Ratio H/M normalized")
df = padua.filters.filter_select_columns_ratio(df, columns=['Proteins','Protein IDs','Gene names','id'] )

df.shape

(5718, 15)

In [9]:
# Check the processed DataFrame
df

Protein IDs            Gene names  Proteins  \
id                                                                     
0                             A0AVT1                  UBA6         1   
1                             A0FGR8                 ESYT2         1   
2                             A0JLT2                 MED19         1   
3                             A0MZ66              KIAA1598         1   
4                             A1A4S6              ARHGAP10         1   
5                             A1L020                 MEX3A         1   
6                             A1L0T0                 ILVBL         1   
7                             A1L157               TSPAN11         1   
8                             A1X283              SH3PXD2B         1   
9                             A2RRP1                  NBAS         1   
10                            A2RU67              KIAA1467         1   
11                            A2VDJ0              KIAA0922         1   
12                            A3KMH1              KIAA0564         1   
13                            A4D1E9               GTPBP10         1   
14                            A4D1P6                 WDR91         1   
15                            A5A3E0                 POTEF         1   
16                            A5PLL7               TMEM189         1   
17                            A5YKK6                 CNOT1         1   
18                     A6NCE7;Q9GZQ8    MAP1LC3B2;MAP1LC3B         2   
19                            A6NDG6                   PGP         1   
20                            A6NDU8               C5orf51         1   
21                            A6NED2                 RCCD1         1   
22                            A6NFI3                ZNF316         1   
23                            A6NFQ2               FAM115C         1   
24                            A6NHL2                TUBAL3         1   
25                            A6NHQ2                 FBLL1         1   
26                            A6NHR9                SMCHD1         1   
27                     A6NIH7;Q13432               UNC119B         2   
28                            A6NJ78               METTL15         1   
29              B7ZAQ6;P0CG08;A6NKF9  GPR89A;GPR89B;GPR89C         3   
...                              ...                   ...       ...   
5791                          Q9Y6D9                MAD1L1         1   
5792            Q9Y6E0;O00506;Q9P289                 STK24         3   
5793                          Q9Y6E2                  BZW2         1   
5794                          Q9Y6G3                MRPL42         1   
5795                          Q9Y6G9              DYNC1LI1         1   
5796                   Q9Y6H1;Q5T1J5       CHCHD2;CHCHD2P9         2   
5797                          Q9Y6I3                  EPN1         1   
5798  Q9Y6I9;REV__P30876;REV__Q9NW08                TEX264         3   
5799                          Q9Y6J0                CABIN1         1   
5800                          Q9Y6J9                 TAF6L         1   
5801                          Q9Y6K1                DNMT3A         1   
5802                          Q9Y6K9                 IKBKG         1   
5803                   Q9Y6M1;Q9NZI8               IGF2BP2         2   
5804                          Q9Y6M4               CSNK1G3         1   
5805                          Q9Y6M5               SLC30A1         1   
5806            Q9Y6M7;Q6U841;Q2Y0W8                SLC4A7         3   
5807                          Q9Y6M9                NDUFB9         1   
5808                          Q9Y6N1                 COX11         1   
5809                          Q9Y6N5                 SQRDL         1   
5810                   Q9Y6N7;Q9HCK4                 ROBO1         2   
5811                          Q9Y6Q2                 STON1         1   
5812                          Q9Y6Q9                 NCOA3         1   
5813                          Q9Y6R0                 NUMBL       

The last two important steps consist first in matching the processed DataFrame with the experimental design, and then filtering the quantified protein-data according to arbitrary criteria. 

In [10]:
%autoreload 2

# Load input file experiment_design.csv
design = pd.read_csv(DESIGN_FILE)

In [11]:
%autoreload 2

# Create DataFrame with experiment annotation 
df = padua.process.build_index_from_design(df, design, types={'Replicate': int})
df.to_csv( os.path.join(SOURCE_FOLDER, BASENAME + '-experiment_design.csv'))
df

Group                                                                                                                            1day/control  \
Replicate                                                                                                                                   1   
id   Protein IDs                    Gene names           Proteins Ratio M/L normalized Ratio H/L normalized Ratio H/M normalized                
0    A0AVT1                         UBA6                 1         0.171335             0.002162            -0.222111                0.233643   
1    A0FGR8                         ESYT2                1        -0.208344             0.248170             0.419215               -0.156820   
2    A0JLT2                         MED19                1         0.120086             0.259062             0.034638               -0.004870   
3    A0MZ66                         KIAA1598             1        -0.281772            -0.991255            -0.541870               -0.295659   
4    A1A4S6                         ARHGAP10             1         0.256528            -0.017621            -0.223761                0.174790   
5    A1L020                         MEX3A                1        -0.355553            -0.679642            -0.388677                     NaN   
6    A1L0T0                         ILVBL                1         0.065159             0.454281             0.330558                0.501516   
7    A1L157                         TSPAN11              1        -0.869254            -0.337120             0.437441                     NaN   
8    A1X283                         SH3PXD2B             1        -0.134097            -0.531573            -0.191771               -0.247782   
9    A2RRP1                         NBAS                 1        -0.106778             0.124593             0.204015               -0.154281   
10   A2RU67                         KIAA1467             1        -0.711441            -0.744971            -0.625111               -0.686104   
11   A2VDJ0                         KIAA0922             1        -0.655831            -1.566229            -0.835265               -0.655831   
12   A3KMH1                         KIAA0564             1        -0.072468             0.167872             0.270708               -0.124078   
13   A4D1E9                         GTPBP10              1        -0.058608             0.086376             0.111966               -0.160443   
14   A4D1P6                         WDR91                1         0.313826             0.252658             0.105008                0.043345   
15   A5A3E0                         POTEF                1        NaN                  NaN                  NaN                           NaN   
16   A5PLL7                         TMEM189              1        -0.071952            -0.156708            -0.257632                0.449957   
17   A5YKK6                         CNOT1                1        -0.119228            -0.238204            -0.139569               -0.118508   
18   A6NCE7;Q9GZQ8                  MAP1LC3B2;MAP1LC3B   2         0.191184            -0.181117            -0.468192                0.249021   
19   A6NDG6                         PGP                  1         0.182184            -0.503814            -0.709269                0.287118   
20   A6NDU8                         C5orf51              1         0.230572             0.019915            -0.132436                0.620023   
21   A6NED2                         RCCD1                1         0.548535             0.318924            -0.424765                0.701150   
22   A6NFI3                         ZNF316               1        -0.476137            -0.419160             0.110230               -0.305414   
23   A6NFQ2                         FAM115C              1        -0.025911            -0.503630            -0.773319               -0.121800   
24   A6NHL2                         TUBAL3               1        NaN                

In [12]:
# Filter DataFrame: At least 1 protein overall time points
%autoreload 2
MINIMUM_VALID_VALUES_PER_GROUP = 1
MINIMUM_VALID_VALUES_LEVELS = [0]
df_all = padua.filters.minimum_valid_values_in_any_group(df, levels=MINIMUM_VALID_VALUES_LEVELS, n=MINIMUM_VALID_VALUES_PER_GROUP, invalid=np.nan)
df_all.to_csv( os.path.join(SOURCE_FOLDER, BASENAME + '-after_valid_values_total.csv'))
df_all.shape

(5459, 9)

In [13]:
# Filter DataFrame: At least 2 biological replicates
%autoreload 2
MINIMUM_VALID_VALUES_PER_GROUP = 2
MINIMUM_VALID_VALUES_LEVELS = [0]
df = padua.filters.minimum_valid_values_in_any_group(df, levels=MINIMUM_VALID_VALUES_LEVELS, n=MINIMUM_VALID_VALUES_PER_GROUP, invalid=np.nan)
df.to_csv( os.path.join(SOURCE_FOLDER, BASENAME + '-after_valid_values_2bioreplicate.csv'))
df.shape

(4785, 9)

## Protein-DataFrame

In [14]:
df.head(5)

Group                                                                                             1day/control  \
Replicate                                                                                                    1   
id Protein IDs Gene names Proteins Ratio M/L normalized Ratio H/L normalized Ratio H/M normalized                
0  A0AVT1      UBA6       1         0.171335             0.002162            -0.222111                0.233643   
1  A0FGR8      ESYT2      1        -0.208344             0.248170             0.419215               -0.156820   
2  A0JLT2      MED19      1         0.120086             0.259062             0.034638               -0.004870   
3  A0MZ66      KIAA1598   1        -0.281772            -0.991255            -0.541870               -0.295659   
6  A1L0T0      ILVBL      1         0.065159             0.454281             0.330558                0.501516   

Group                                                                                                        \
Replicate                                                                                                 2   
id Protein IDs Gene names Proteins Ratio M/L normalized Ratio H/L normalized Ratio H/M normalized             
0  A0AVT1      UBA6       1         0.171335             0.002162            -0.222111             0.177726   
1  A0FGR8      ESYT2      1        -0.208344             0.248170             0.419215            -0.225413   
2  A0JLT2      MED19      1         0.120086             0.259062             0.034638                  NaN   
3  A0MZ66      KIAA1598   1        -0.281772            -0.991255            -0.541870            -0.240674   
6  A1L0T0      ILVBL      1         0.065159             0.454281             0.330558             0.045862   

Group                                                                                                        \
Replicate                                                                                                 3   
id Protein IDs Gene names Proteins Ratio M/L normalized Ratio H/L normalized Ratio H/M normalized             
0  A0AVT1      UBA6       1         0.171335             0.002162            -0.222111             0.139731   
1  A0FGR8      ESYT2      1        -0.208344             0.248170             0.419215            -0.239941   
2  A0JLT2      MED19      1         0.120086             0.259062             0.034638             0.208892   
3  A0MZ66      KIAA1598   1        -0.281772            -0.991255            -0.541870            -0.292458   
6  A1L0T0      ILVBL      1         0.065159             0.454281             0.330558            -0.005102   

Group                                                                                             3days/1day  \
Replicate                                                                                                  1   
id Protein IDs Gene names Proteins Ratio M/L normalized Ratio H/L normalized Ratio H/M normalized              
0  A0AVT1      UBA6       1         0.171335             0.002162            -0.222111              0.271187   
1  A0FGR8      ESYT2      1        -0.208344             0.248170             0.419215              0.163112   
2  A0JLT2      MED19      1         0.120086             0.259062             0.034638              0.007913   
3  A0MZ66      KIAA1598   1        -0.281772            -0.991255            -0.541870              0.101381   
6  A1L0T0      ILVBL      1         0.065159             0.454281             0.330558              0.153027   

Group                                                                                                        \
Replicate                                                                                                 2   
id Protein IDs Gene names Proteins Ratio M/L normalized Ratio H/L normalized Ratio H/M normalized             
0  A0AVT1      UBA6       1         0.171335             0.002162            -0

Finally both pickle and csv processed files can be exported for he next analysis step.

In [15]:
# Export output for next analysis
df.to_csv( os.path.join(SOURCE_FOLDER, BASENAME + '-processed.csv'))
df.to_pickle( os.path.join(SOURCE_FOLDER, BASENAME + '-processed.pickle'))